In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

In [ ]:
base = os.path.abspath('.')
allpoints = os.path.join(base, "dataset/allpoints.csv")


In [ ]:
df = pd.read_csv(allpoints)
df.head()

In [ ]:
# get one time stamp data
timestamps = list(set(df['date'].tolist()))
sampletime = timestamps[0]
sampletime

In [ ]:
def assign_int(x):
    if x == 'water':
        return 1
    else:
        return 0

def assign_class(x, threshold):
    
    if float(x) >= float(threshold):
        return 0
    else:
        return 1

def assign_intall(x):
    if x == 'water':
        return 0
    elif x == 'farmland':
        return 1
    elif x == 'bare':
        return 2

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()


In [ ]:
df['ld_int'] = df['ld_type'].apply(lambda x: assign_int(x))
testrf_df = df.loc[df['date']==sampletime][['BRIGHTI', 'BSI', 'GNDVI', 'NDDI', 'NDPI', 'NDVI', 'NDWI','RedEdge1', 'SRRE', 'SWI', 'ld_int']]

In [ ]:
testrf_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt


X = testrf_df.iloc[:, :-1]
Y = testrf_df.iloc[:, -1]

X = X.values
y = Y.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1693)
scale_X = StandardScaler()
X_train = scale_X.fit_transform(X_train)
X_test = scale_X.fit_transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

rt_classifier = RandomForestClassifier(n_estimators=1000, random_state=1693, max_depth=5)
rt_classifier.fit(X_train, y_train)


In [ ]:
rt_pred = rt_classifier.predict(X_test)
rt_cm = confusion_matrix(y_test, rt_pred)
rt_cm

In [ ]:
print_cm(rt_cm, ['0', '1', '2'])